In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import folium
from geopy.distance import geodesic

In [38]:
gombe = pd.read_csv('GOMBE_crosschecked.csv')
gombe.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,Latitude,Longitude
0,GOMBE,AKKO,AKKO,15-01-01-001,"LAFIYAWO, KOFAR JAURO",218,622,True,-1,False,...,False,False,UNKNOWN,52,8,142,2,https://docs.inecelectionresults.net/elections...,10.290024,11.064610
1,GOMBE,AKKO,AKKO,15-01-01-002,"MARORI / WUI-WUI, KOFAR JAURO",166,547,True,-1,False,...,False,False,UNKNOWN,59,1,90,10,https://docs.inecelectionresults.net/elections...,10.285326,10.937789
2,GOMBE,AKKO,AKKO,15-01-01-003,"AKKO, AKKO PRI. SCH.",351,707,True,-1,False,...,False,False,UNKNOWN,97,1,212,1,https://docs.inecelectionresults.net/elections...,10.289834,10.967454
3,GOMBE,AKKO,AKKO,15-01-01-004,"S/YAKI/J. HAMMA, AKKO PRI. SCH.",209,462,True,-1,False,...,False,False,UNKNOWN,70,0,137,0,https://docs.inecelectionresults.net/elections...,10.296131,10.944418
4,GOMBE,AKKO,AKKO,15-01-01-007,"JALINGO, KOFAR JAURO",231,372,True,-1,False,...,False,False,UNKNOWN,60,1,147,1,https://docs.inecelectionresults.net/elections...,10.910794,11.320626


In [14]:
def find_neighbors_PU_Name(row, df, radius=1):
    neighbours_PU_name = []
    #neighbours_PU_code = []
    for _, polling_unit in df.iterrows():
        distance = geodesic((polling_unit['Latitude'], polling_unit['Longitude']), (row['Latitude'], row['Longitude'])).km
        if (distance <= radius) and (row['PU-Code'] != polling_unit['PU-Code']):
            neighbours_PU_name.append(polling_unit['PU-Name'])
            #neighbours_PU_code.append(polling_unit['PU-Code'])
    return neighbours_PU_name

def find_neighbors_PU_Code(row, df, radius=1):
    #neighbours_PU_name = []
    neighbours_PU_code = []
    for _, polling_unit in df.iterrows():
        distance = geodesic((polling_unit['Latitude'], polling_unit['Longitude']), (row['Latitude'], row['Longitude'])).km
        if (distance <= radius) and (row['PU-Code'] != polling_unit['PU-Code']):
            #neighbours_PU_name.append(polling_unit['PU-Name'])
            neighbours_PU_code.append(polling_unit['PU-Code'])
    return neighbours_PU_code

In [39]:
gombe['neighbours-PU-Name'] = gombe.apply(find_neighbors_PU_Name, args=(gombe, 2), axis=1)
gombe['neighbours-PU-Code'] = gombe.apply(find_neighbors_PU_Code, args=(gombe, 2), axis=1)

In [16]:
# Function to calculate outlier scores
def calculate_outlier_scores(row, df, party):
    outlier_score = []
    neighbors = row['neighbours-PU-Code']
    
    if neighbors==[]:
        return 0
    else:
        for _, polling_unit in df.iterrows():
            if polling_unit['PU-Code'] in neighbors:
                difference = abs(row[party] - polling_unit[party])
                outlier_score.append(difference)
        return np.mean(outlier_score)

In [40]:
gombe['outlier_score_APC'] = gombe.apply(calculate_outlier_scores, args=(gombe, 'APC'), axis=1)
gombe['outlier_score_LP'] = gombe.apply(calculate_outlier_scores, args=(gombe, 'LP'), axis=1)
gombe['outlier_score_PDP'] = gombe.apply(calculate_outlier_scores, args=(gombe, 'PDP'), axis=1)
gombe['outlier_score_NNPP'] = gombe.apply(calculate_outlier_scores, args=(gombe, 'NNPP'), axis=1)

In [41]:
gombe.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,NNPP,Results_File,Latitude,Longitude,neighbours-PU-Name,neighbours-PU-Code,outlier_score_APC,outlier_score_LP,outlier_score_PDP,outlier_score_NNPP
0,GOMBE,AKKO,AKKO,15-01-01-001,"LAFIYAWO, KOFAR JAURO",218,622,True,-1,False,...,2,https://docs.inecelectionresults.net/elections...,10.290024,11.064610,[TIKEN SHANU I],[15-11-08-052],39.000000,8.000000,108.000000,5.000000
1,GOMBE,AKKO,AKKO,15-01-01-002,"MARORI / WUI-WUI, KOFAR JAURO",166,547,True,-1,False,...,10,https://docs.inecelectionresults.net/elections...,10.285326,10.937789,"[S/YAKI/J. HAMMA, AKKO PRI. SCH., DUNAMARI - ...","[15-01-01-004, 15-01-01-009, 15-01-01-011, 15-...",31.833333,1.000000,71.000000,8.666667
2,GOMBE,AKKO,AKKO,15-01-01-003,"AKKO, AKKO PRI. SCH.",351,707,True,-1,False,...,1,https://docs.inecelectionresults.net/elections...,10.289834,10.967454,"[JAN GERAWO - G.J. SHEHU, K. J. SHEHU, GUDDE S...","[15-01-01-018, 15-01-01-019, 15-01-01-026, 15-...",48.727273,0.909091,106.272727,1.818182
3,GOMBE,AKKO,AKKO,15-01-01-004,"S/YAKI/J. HAMMA, AKKO PRI. SCH.",209,462,True,-1,False,...,0,https://docs.inecelectionresults.net/elections...,10.296131,10.944418,"[MARORI / WUI-WUI, KOFAR JAURO, DUNAMARI - BO...","[15-01-01-002, 15-01-01-009, 15-01-01-011, 15-...",32.083333,0.250000,61.833333,2.166667
4,GOMBE,AKKO,AKKO,15-01-01-007,"JALINGO, KOFAR JAURO",231,372,True,-1,False,...,1,https://docs.inecelectionresults.net/elections...,10.910794,11.320626,[],[],0.000000,0.000000,0.000000,0.000000


In [42]:
gombe['no. of neighbors'] = gombe['neighbours-PU-Code'].apply(lambda x: len(x))

In [43]:
gombe['outlier_score_APC'] = gombe['outlier_score_APC'].round(0).astype('int64')
gombe['outlier_score_LP'] = gombe['outlier_score_LP'].round(0).astype('int64')
gombe['outlier_score_PDP'] = gombe['outlier_score_PDP'].round(0).astype('int64')
gombe['outlier_score_NNPP'] = gombe['outlier_score_NNPP'].round(0).astype('int64')

In [49]:
gombe.columns = ['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Accredited_Voters',
       'Registered_Voters', 'Results_Found', 'Transcription_Count',
       'Result_Sheet_Stamped', 'Result_Sheet_Corrected',
       'Result_Sheet_Invalid', 'Result_Sheet_Unclear', 'Result_Sheet_Unsigned',
       'APC', 'LP', 'PDP', 'NNPP', 'Results_File', 'Latitude', 'Longitude',
       'neighbors-PU-Name', 'neighbors-PU-Code', 'outlier_score_APC',
       'outlier_score_LP', 'outlier_score_PDP', 'outlier_score_NNPP',
       'no. of neighbors']

In [50]:
gombe_outlier_score = gombe[['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name',
       'APC', 'LP', 'PDP', 'NNPP', 'Latitude', 'Longitude',
       'neighbors-PU-Name', 'no. of neighbors', 'outlier_score_APC',
       'outlier_score_LP', 'outlier_score_PDP', 'outlier_score_NNPP']]
gombe.to_excel('GOMBE_with_outlier_score_2km.xlsx', index=False)
#gombe_outlier_score.to_excel('GOMBE_with_outlier_scores.xlsx', index=False)

In [46]:
outlier_pu_code = ['15-06-06-027','15-07-07-011','15-05-01-020', '15-05-06-008', '15-07-05-027', '15-08-01-005', '15-11-09-003', '15-07-05-015', '15-04-07-004', '15-07-07-015']
outliers = gombe[gombe['PU-Code'].isin(outlier_pu_code)]

In [53]:
m = folium.Map(location=[10.277891902212696, 11.171844717868716], zoom_start=9)
for _, row in gombe.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        popup=f"<strong>{row['PU-Name']}</strong><br>APC: {row['outlier_score_APC']}<br>LP: {row['outlier_score_LP']}<br>PDP: {row['outlier_score_PDP']}<br>NNPP: {row['outlier_score_NNPP']}"
    ).add_to(m)
for _, row in outliers.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"<strong>{row['PU-Name']}</strong><br>APC: {row['outlier_score_APC']}<br>LP: {row['outlier_score_LP']}<br>PDP: {row['outlier_score_PDP']}<br>NNPP: {row['outlier_score_NNPP']}",
        icon = folium.Icon(color='red')
    ).add_to(m)
m.save('gombe.html')
m

In [52]:
gombe_outlier_score.head(10)

,State,LGA,Ward,PU-Code,PU-Name,APC,LP,PDP,NNPP,Latitude,Longitude,neighbors-PU-Name,no. of neighbors,outlier_score_APC,outlier_score_LP,outlier_score_PDP,outlier_score_NNPP
0,GOMBE,AKKO,AKKO,15-01-01-001,"LAFIYAWO, KOFAR JAURO",52,8,142,2,10.290024,11.064610,[TIKEN SHANU I],1,39,8,108,5
1,GOMBE,AKKO,AKKO,15-01-01-002,"MARORI / WUI-WUI, KOFAR JAURO",59,1,90,10,10.285326,10.937789,"[S/YAKI/J. HAMMA, AKKO PRI. SCH., DUNAMARI - ...",6,32,1,71,9
2,GOMBE,AKKO,AKKO,15-01-01-003,"AKKO, AKKO PRI. SCH.",97,1,212,1,10.289834,10.967454,"[JAN GERAWO - G.J. SHEHU, K. J. SHEHU, GUDDE S...",11,49,1,106,2
3,GOMBE,AKKO,AKKO,15-01-01-004,"S/YAKI/J. HAMMA, AKKO PRI. SCH.",70,0,137,0,10.296131,10.944418,"[MARORI / WUI-WUI, KOFAR JAURO, DUNAMARI - BO...",12,32,0,62,2
4,GOMBE,AKKO,AKKO,15-01-01-007,"JALINGO, KOFAR JAURO",60,1,147,1,10.910794,11.320626,[],0,0,0,0,0
5,GOMBE,AKKO,AKKO,15-01-01-008,"KOMBANI, KOFAR JAURO",69,0,163,3,10.048283,11.014521,[],0,0,0,0,0
6,GOMBE,AKKO,AKKO,15-01-01-009,"DUNAMARI - BOGGAGEL, LAWANTI PRI. SCH.",20,0,181,2,10.296131,10.944418,"[MARORI / WUI-WUI, KOFAR JAURO, S/YAKI/J. HAMM...",12,33,0,94,2
7,GOMBE,AKKO,AKKO,15-01-01-010,"JALINGO / AKKO, K. S. LAWANTI",24,0,71,4,10.258020,11.192397,"[HAMMADU KAFI YAMMA GINDIN BOREHOLE, GADAN TAB...",7,31,8,30,3
8,GOMBE,AKKO,AKKO,15-01-01-011,"DOLLI JILIM - DOLLI KUTURE, DOLLI PRI. SCH.",68,0,86,0,10.296131,10.944418,"[MARORI / WUI-WUI, KOFAR JAURO, S/YAKI/J. HAMM...",12,31,0,48,2
9,GOMBE,AKKO,AKKO,15-01-01-012,"DON DORIYA - DOLLI NGALDA, DOLLI PRI. SCH.",87,0,227,6,10.296131,10.944418,"[MARORI / WUI-WUI, KOFAR JAURO, S/YAKI/J. HAMM...",12,46,0,136,5
